# Chapter 2: Opening and Reading NetCDF files using STAC
**Creators**: Ann Windnagel and Robyn Marowitz

**Affiliation**: [National Snow And Ice Data Center](https://nsidc.org/home)

## Learning Objectives

In this notebook, you will learn how to open and read NetCDF files using STAC and explore the features and vairables of the STAC Collection and Items.

### Import relevant packages
We will use [pystac](https://pystac.readthedocs.io/en/stable/) to explore the sea ice CDR STAC collection and items and their details. [NetCDF4](https://pystac.readthedocs.io/en/stable/) is a Python interface to the netCDF C library that is used to read in information from NetCDF files. [Xarray](https://docs.xarray.dev/en/stable/) makes working with labelled multi-dimensional arrays, such as NetCDF, in Python simple and efficient.


In [1]:
import xarray as xr
import numpy
import pandas
import packaging
import aiohttp
import fsspec
import pystac
import netCDF4
from netCDF4 import Dataset
import h5netcdf

## STAC Collections

[A STAC Collection](https://github.com/radiantearth/stac-spec/tree/master/collection-spec) object is used to describe a group of reltaed Items.  

To begin, we will read in the sea ice CDR STAC `collection.json` file and assign it to a variable `collection` so that we can explore it using `pystac`.

In [3]:
# Specify the URL to the sea ice CDR collection.json file on NSIDC servers
collection_url = 'https://noaadata.apps.nsidc.org/NOAA/G02202_V4/stac/collection.json'
# Read file and assign to a variable
collection = pystac.Collection.from_file(collection_url)

## Explore Collection Information

In the following cells, you will look at how the Collection is organized. Using the `describe` method, you can see all of the `Items` listed. In the `Item id`, there is already a wealth of information. Note that `nh` means northern hemisphere and `sh` means southern hemisphere.

The sea ice CDR data spans 1978 through the present year. You will see an item listed for each year and hemipshere for the daily files, plus one monthly file for each hemisphere that contains the whole time series.

In [4]:
# List all the Items and their Item IDs
collection.describe()

* <Collection id=noaa-cdr-sea-ice-concentration>
  * <Item id=seaice_conc_daily_sh_2023_v04r00>
  * <Item id=seaice_conc_daily_sh_2022_v04r00>
  * <Item id=seaice_conc_daily_sh_2021_v04r00>
  * <Item id=seaice_conc_daily_sh_2020_v04r00>
  * <Item id=seaice_conc_daily_sh_2019_v04r00>
  * <Item id=seaice_conc_daily_sh_2018_v04r00>
  * <Item id=seaice_conc_daily_sh_2017_v04r00>
  * <Item id=seaice_conc_daily_sh_2016_v04r00>
  * <Item id=seaice_conc_daily_sh_2015_v04r00>
  * <Item id=seaice_conc_daily_sh_2014_v04r00>
  * <Item id=seaice_conc_daily_sh_2013_v04r00>
  * <Item id=seaice_conc_daily_sh_2012_v04r00>
  * <Item id=seaice_conc_daily_sh_2011_v04r00>
  * <Item id=seaice_conc_daily_sh_2010_v04r00>
  * <Item id=seaice_conc_daily_sh_2009_v04r00>
  * <Item id=seaice_conc_daily_sh_2008_v04r00>
  * <Item id=seaice_conc_daily_sh_2005_v04r00>
  * <Item id=seaice_conc_daily_sh_2004_v04r00>
  * <Item id=seaice_conc_daily_sh_2003_v04r00>
  * <Item id=seaice_conc_daily_sh_2002_v04r00>
  * <Item i

### TBD Look into pretty print because of the way the end of the line doesn't wrap nicely?

In [5]:
# Print the title and description of the collection
print('Title: ',collection.title)
print('Description: ',collection.description)

Title:  Sea Ice Concentration CDR
Description:  The Sea Ice Concentration Climate Data Record (CDR) provides a consistent daily and monthly time series of sea ice concentrations for both the north and south Polar Regions on a 25 km x 25 km grid. These data can be used to estimate how much of the ocean surface is covered by ice, and monitor changes in sea ice concentration. The CDR combines concentration estimates using two algorithms developed at the NASA Goddard Space Flight Center (GSFC). Gridded brightness temperatures acquired from a number of Defense Meteorological Satellite Program (DMSP) passive microwave radiometers provide the necessary input to produce the dataset.


## Get all items 
Extract all the Items from the collection into a variable `items`

In [11]:
# Extract items from the collection and print out how many there are
items = list(collection.get_all_items())
print(f"Number of items: {len(items)}")

Number of items: 92


'seaice_conc_daily_sh_2023_v04r00'

## Explore an item

Select one Item using the Item ID and explore its metadata. In this case, you will explore the 2023 dialy southern hemisphere Item. Feel free to explore other years and hemispheres.

In [12]:
item_id = items[0].id
daily_sh_2023_item = collection.get_item(item_id, recursive=True)

In [13]:
daily_sh_2023_item.geometry

{'type': 'Polygon',
 'coordinates': [[[180.0, -90.0],
   [180.0, -41.45],
   [-180.0, -41.45],
   [-180.0, -90.0],
   [180.0, -90.0]]]}

In [8]:
daily_sh_2023_item.bbox

[-180.0, -90.0, 180.0, -41.45]

In [9]:
daily_sh_2023_item.properties['start_datetime']

'2023-01-01T00:00:00Z'

In [10]:
daily_sh_2023_item.properties['end_datetime']

'2023-12-31T23:59:59Z'

In [12]:
daily_sh_2023_item.set_datetime(daily_sh_2023_item.properties['start_datetime'])

In [13]:
daily_sh_2023_item.datetime

'2023-01-01T00:00:00Z'

In [14]:
daily_sh_2023_item.properties

{'start_datetime': '2023-01-01T00:00:00Z',
 'end_datetime': '2023-12-31T23:59:59Z',
 'noaa_cdr:interval': 'daily',
 'processing:level': 'L3',
 'proj:epsg': None,
 'proj:wkt2': 'PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["unknown",ELLIPSOID["unknown",6378273,298.279411123064,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8901]]],CONVERSION["unknown",METHOD["Polar Stereographic (variant B)",ID["EPSG",9829]],PARAMETER["Latitude of standard parallel",-70,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8832]],PARAMETER["Longitude of origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8833]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["(E)",north,MERIDIAN[90,ANGLEUNIT["degree",0.0174532925199433,ID["EPSG",9122]]],ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["(N)",north,MERIDIAN[0,ANGLEUNIT["degree",0

## Read NetCDF

In [14]:
# Access the NetCDF asset and see the components
daily_sh_2023_item.assets['netcdf']

<Asset href=https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc>

In [15]:
netcdf_url = daily_sh_2023_item.assets['netcdf'].href
netcdf_url

'https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc'

In [16]:
fs = fsspec.filesystem('https')
ds = xr.open_dataset(fs.open(netcdf_url))
ds

<xarray.Dataset> Size: 1GB
Dimensions:                      (tdim: 365, y: 332, x: 316)
Coordinates:
    time                         (tdim) datetime64[ns] 3kB ...
    xgrid                        (x) float32 1kB ...
    ygrid                        (y) float32 1kB ...
Dimensions without coordinates: tdim, y, x
Data variables:
    cdr_seaice_conc              (tdim, y, x) float32 153MB ...
    nsidc_bt_seaice_conc         (tdim, y, x) float32 153MB ...
    nsidc_nt_seaice_conc         (tdim, y, x) float32 153MB ...
    projection                   |S1 1B ...
    qa_of_cdr_seaice_conc        (tdim, y, x) float32 153MB ...
    spatial_interpolation_flag   (tdim, y, x) float32 153MB ...
    stdev_of_cdr_seaice_conc     (tdim, y, x) float32 153MB ...
    temporal_interpolation_flag  (tdim, y, x) float32 153MB ...
    latitude                     (y, x) float32 420kB ...
    longitude                    (y, x) float32 420kB ...
Attributes: (12/39)
    Conventions:               CF-1.6, ACDD-1.3
    title:                     NOAA/NSIDC Climate Data Record of Passive Micr...
    references:                Comiso, J. C., and F. Nishio. 2008. Trends in ...
    comment:                   none
    program:                   NOAA Climate Data Record Program
    cdr_variable:              cdr_seaice_conc
    ...                        ...
    acknowledgment:            This project was supported in part by a grant ...
    platform:                  DMSP 5D-3/F17 > Defense Meteorological Satelli...
    sensor:                    SSMI/S > Special Sensor Microwave Imager/Sounder
    date_created:              2023-09-26T20:17:19Z
    time_coverage_start:       2023-01-01T00:00:00Z
    time_coverage_end:         2023-12-31T23:59:59Z

In [17]:
# Take a look at the variable keys for the NetCDF `dataset`
ds.variables.keys()

KeysView(Frozen({'cdr_seaice_conc': <xarray.Variable (tdim: 365, y: 332, x: 316)> Size: 153MB
[38292880 values with dtype=float32]
Attributes:
    long_name:            NOAA/NSIDC Climate Data Record of Passive Microwave...
    standard_name:        sea_ice_area_fraction
    units:                1
    flag_values:          [251 252 253 254 255]
    flag_meanings:        pole_hole lakes coastal land_mask missing_data
    datum:                +ellps=urn:ogc:def:crs:EPSG::4326
    grid_mapping:         projection
    reference:            https://nsidc.org/data/g02202/versions/4/
    ancillary_variables:  stdev_of_cdr_seaice_conc qa_of_cdr_seaice_conc
    valid_range:          [  0 100]
    cell_methods:         tdim: mean, 'nsidc_bt_seaice_conc': <xarray.Variable (tdim: 365, y: 332, x: 316)> Size: 153MB
[38292880 values with dtype=float32]
Attributes:
    long_name:      Passive Microwave Daily Southern Hemisphere Sea Ice Conce...
    standard_name:  sea_ice_area_fraction
    units:   

In [18]:
# Pull out sea ice concentration variable
cdr_seaice_conc = ds.variables['cdr_seaice_conc']
cdr_seaice_conc

<xarray.Variable (tdim: 365, y: 332, x: 316)> Size: 153MB
[38292880 values with dtype=float32]
Attributes:
    long_name:            NOAA/NSIDC Climate Data Record of Passive Microwave...
    standard_name:        sea_ice_area_fraction
    units:                1
    flag_values:          [251 252 253 254 255]
    flag_meanings:        pole_hole lakes coastal land_mask missing_data
    datum:                +ellps=urn:ogc:def:crs:EPSG::4326
    grid_mapping:         projection
    reference:            https://nsidc.org/data/g02202/versions/4/
    ancillary_variables:  stdev_of_cdr_seaice_conc qa_of_cdr_seaice_conc
    valid_range:          [  0 100]
    cell_methods:         tdim: mean

In [23]:
ds.platform

'DMSP 5D-3/F17 > Defense Meteorological Satellite Program-F17'

In [26]:
ds.sensor

'SSMI/S > Special Sensor Microwave Imager/Sounder'